In [1]:
cd ~/SageMaker/docker_test_folder

/home/ec2-user/SageMaker/docker_test_folder


In [2]:
! pwd

/home/ec2-user/SageMaker/docker_test_folder


In [3]:
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
! docker build -t pytorch-extended-container-test .

Sending build context to Docker daemon  14.34kB
Step 1/5 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.1-cpu-py36-ubuntu16.04
1.5.1-cpu-py36-ubuntu16.04: Pulling from pytorch-training

a89234b4: Pulling fs layer 
26c6b9c9: Pulling fs layer 
bf18aa40: Pulling fs layer 
c688ebe3: Pulling fs layer 
d99041c3: Pulling fs layer 
5b8a173b: Pulling fs layer 
3555f0d1: Pulling fs layer 
20dcac6d: Pulling fs layer 
edcfbd21: Pulling fs layer 
32f60d8c: Pulling fs layer 
26599d36: Pulling fs layer 
659108d7: Pulling fs layer 
d26ed72a: Pulling fs layer 
f55933a3: Pulling fs layer 
Digest: sha256:b3aac61299d88695b39ed947c87c16cc2f8fb3a985dc97634ae785d888256b542KExtracting  97.48MB/839.1MBExtracting  37.88MB/214.1MB
Status: Downloaded newer image for 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.1-cpu-py36-ubuntu16.04
 ---> 6177bf009b27
Step 2/5 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Running in 8ba581dd64b7
Removing intermediate container 8ba581dd64b7

In [5]:
%%sh

# Specify an algorithm name
algorithm_name=pytorch-extended-container-test

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "${region}"
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

# docker push ${fullname}

us-east-1
Login Succeeded

Step 1/5 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.1-cpu-py36-ubuntu16.04
 ---> 6177bf009b27
Step 2/5 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 617f3f17e92c
Step 3/5 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> f1f21f8306c2
Step 4/5 : COPY cifar10.py /opt/ml/code/cifar10.py
 ---> Using cache
 ---> bafd71de3df7
Step 5/5 : ENV SAGEMAKER_PROGRAM cifar10.py
 ---> Using cache
 ---> df13c7fda85d
Successfully built df13c7fda85d
Successfully tagged pytorch-extended-container-test:latest


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [10]:
region

NameError: name 'region' is not defined